## Furniture Company -  Version 6

**This version has scaled down values to enhance the experience of the second game. It is also adjusted to provide integer costs, if items should be split into parts.**

Aspects to consider when working with different combinations of variables:

### Variation among items

+ Looking at variation coefficients among the items for every month, a low value indicates that different furniture is built is distributed similarly.
+ At a higher value results indicate the concentration on one item mainly built at one month.

### Variation for each item over a year

+ Higher values indicate, that the months in which these items are built vary more, e.g they are built in just one month throughout the year.
+ Low values show that the distribution of items throughout the year is balanced.
+ This enables for example one item to be mainly built in every month, while other items can be relevant just in some months.

+ Values < 0.5 makes the items appear regularly
+ Values of >= 1 makes them appear in 8 of 12 months with different quantity.

+ A useful criterion would be values smaller than 1.1 and greater than 0.65.

## Model

In [1]:
# This is used to import the modules from the experiment folder
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
# First the model builder functionality needs to be imported.
from model_builder import Item, ModelBuilder
from pyomo.core.base import (Var, value, Objective)
import numpy as np

In [2]:
def reorder_list(ls, indices):
    """Reoders a list by the provided list of indices.
    The list of indices provides the new order by defining which element shall be put 
    at the given place.
    
    Args:
        ls (list): the list that should be reordered.
        indices (list): a list of indices with the same length as ls.
        
    Example: 
    my_list = [1,2,3]
    reorder_list(my_list, [2,0,1]) # --> my_list = [3,1,2]
    """
    ls = [ls[x] for x in indices]
    return ls

In [3]:
# These values are needed to build the model
factor_resources = 1
indices = [6,1,4,3,2,5,0,10,8,9,7,11] # Lists have been reordered by request. Remove the method calls if you want to
# change the order freely. It just could save time if you want to change specific months.
# define the profit per month for each item:
chair_profit =    reorder_list([2,3,3,2,2,1,2,3,2,2,2,2], indices)
table_profit =    reorder_list([4,4,4,5,4,3,4,5,4,4,3,3], indices)
bed_profit =      reorder_list([7,7,5,5,3,5,4,4,5,4,5,6], indices)
bookcase_profit = reorder_list([7,8,10,8,6,7,7,8,10,9,7,10], indices)
 


# define the items:
chair = Item(costs_wood=4,costs_metal=1,costs_time_one=4,costs_time_two=1,profit=chair_profit)
table = Item(2,5,4,6,table_profit)
bed = Item(4,3,3,4, bed_profit)
bookcase = Item(5,7,5,3,bookcase_profit)
# Making calculations for one year
months = 12
# specifying available materials
avail_hours_a = 26*factor_resources
avail_hours_b = 30*factor_resources
avail_hours_c = 23*factor_resources
avail_hours_d = 26*factor_resources

avail_wood = list(map(lambda i: round(i/8), reorder_list([200, 300, 457, 372, 322, 432, 413, 406, 377, 412, 395, 366],indices)))
avail_metal = list(map(lambda i: round(i/8), reorder_list([463, 168, 773, 380, 353, 391, 392, 473, 370, 350, 351, 293], indices)))

In [4]:
# Set up the model builder: 
model_builder = ModelBuilder(months=months, avail_wood=avail_wood, avail_metal=avail_metal
                            , avail_hours_a=avail_hours_a,avail_hours_b=avail_hours_b
                            , avail_hours_c=avail_hours_c,avail_hours_d=avail_hours_d
                            , chair=chair,table=table, bed=bed, bookcase=bookcase, continuous = True)
# build the model:
model = model_builder.build_model()


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpikajuuoj.glpk.raw --wglp /tmp/tmpxvs7c3db.glpk.glp --cpxlp
 /tmp/tmpsubury69.pyomo.lp
Reading problem data from '/tmp/tmpsubury69.pyomo.lp'...
73 rows, 49 columns, 193 non-zeros
516 lines were read
Writing problem data to '/tmp/tmpxvs7c3db.glpk.glp'...
439 lines were written
GLPK Simplex Optimizer, v4.65
73 rows, 49 columns, 193 non-zeros
Preprocessing...
72 rows, 48 columns, 192 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  7.000e+00  ratio =  7.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 72
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (48)
*    46: obj =   6.722036733e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.1 Mb (125587 bytes)
Writing basic solution to '/tmp/tmpikajuuoj.glpk.raw'...
131 lines were written


In [16]:
def solution(m):
    x = []
    for v in m.component_objects(Var, active=True):
        l = []
        for index in v:
            l.append(np.round(v[index].value,2))
        x.append(l)
    return x

In [17]:
value(model.cost)

672.2036732874967

In [18]:
solution(model)

[[5.55, 5.75, 0.0, 5.64, 0.0, 5.55, 3.75, 5.55, 0.0, 0.0, 0.0, 5.55],
 [1.27, 0.0, 3.85, 1.14, 4.6, 1.27, 0.0, 1.27, 4.6, 4.6, 4.6, 1.27],
 [4.24, 0.0, 2.63, 4.14, 4.7, 4.24, 5.0, 2.99, 1.82, 1.32, 4.7, 1.58],
 [2.26, 3.75, 3.87, 2.36, 1.8, 2.26, 0.0, 3.51, 4.68, 5.18, 1.8, 3.58]]

In [ ]:
beds: [5.5, 5.8, 0.0, 5.6,   0.0, 5.5, 3.8, 5.5,   0.0, 0.0, 0.0, 5.5],
book: [1.3, 0.0, 3.9, 1.1,   4.6, 1.3, 0.0, 1.3,   4.6, 4.6, 4.6, 1.3],
table [4.2, 0.0, 2.6, 4.1,   4.7, 4.2, 5.0, 3.0,   1.8, 1.3, 4.7, 1.6],
chair [2.3, 3.8, 3.9, 2.4,   1.8, 2.3, 0.0, 3.5,   4.7, 5.2, 1.8, 3.6]